In [2]:
# Function to rearrange bits according to a pattern
def permute(bits, pattern):
    return ''.join(bits[i - 1] for i in pattern)

# Function for circular left shift
def left_shift(bits, n):
    return bits[n:] + bits[:n]


In [3]:
permute("1010000010", [3,5,2,7,4,10,1,9,8,6])
# Output: '1000001100'


'1000001100'

In [13]:
def key_generation(key):
    # Step 1: P10
    P10 = [3,5,2,7,4,10,1,9,8,6]
    P8  = [6,3,7,4,8,5,10,9]
    
    key = permute(key, P10)
    left, right = key[:5], key[5:]
    
    # Left shift by 1
    left, right = left_shift(left, 1), left_shift(right, 1)
    K1 = permute(left + right, P8)
    
    # Left shift by 2
    left, right = left_shift(left, 2), left_shift(right, 2)
    K2 = permute(left + right, P8)
    
    return K1, K2


In [15]:
key = "1010000010"
K1, K2 = key_generation(key)
print("K1:", K1, "K2:", K2)


K1: 10100100 K2: 01000011


In [16]:
def fk(bits, key):
    EP = [4,1,2,3,2,3,4,1]
    S0 = [
        [1,0,3,2],
        [3,2,1,0],
        [0,2,1,3],
        [3,1,3,2]
    ]
    S1 = [
        [0,1,2,3],
        [2,0,1,3],
        [3,0,1,0],
        [2,1,0,3]
    ]
    P4 = [2,4,3,1]

    left, right = bits[:4], bits[4:]
    
    # Step 1: Expand and permute
    right_expanded = permute(right, EP)
    
    # Step 2: XOR with key
    xor_result = ''.join(str(int(a) ^ int(b)) for a,b in zip(right_expanded, key))
    
    # Step 3: Split into 2 halves for S-boxes
    left_xor, right_xor = xor_result[:4], xor_result[4:]
    
    # Step 4: S-box substitution
    def sbox_lookup(bits, sbox):
        row = int(bits[0] + bits[3], 2)
        col = int(bits[1] + bits[2], 2)
        return format(sbox[row][col], '02b')
    
    sbox_output = sbox_lookup(left_xor, S0) + sbox_lookup(right_xor, S1)
    
    # Step 5: XOR with left half and right half
    left_result = ''.join(str(int(a) ^ int(b)^int(c)) for a,b,c in zip(left, sbox_output,right))
    
    # Step 6: Combine halves
    return left_result + right


In [ ]:
def encrypt(plaintext, key):
    IP = [2,6,3,1,4,8,5,7]
    IP_inv = [4,1,3,5,7,2,8,6]
    
    # Generate keys
    K1, K2 = key_generation(key)
    
    # Step 1: Initial Permutation
    bits = permute(plaintext, IP)
    #K1="10100010"
    # Step 2: Round 1
    bits = fk(bits, K1)
    
    # Step 3: Swap halves
    bits = bits[4:] + bits[:4]
    
    # Step 4: Round 2
    bits = fk(bits, K2)
    
    # Step 5: Inverse IP
    ciphertext = permute(bits, IP_inv)
    return ciphertext


In [20]:
plaintext = "10100010"
key = "1010000010"
cipher = encrypt(plaintext, key)
print("Ciphertext:", cipher)


Ciphertext: 00100010
